In [173]:
# Desabilita warnings
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
# train = pd.read_csv('train.csv')
# test = pd.read_csv('test.csv')
train = pd.read_csv('datasets/titanic/train.csv')
test = pd.read_csv('datasets/titanic/test.csv')
X = train[list(test.columns)]
y = train[train.columns[~train.columns.isin(test.columns)]]


In [174]:
X['Embarked'].value_counts().idxmax()

'S'

In [175]:
from sklearn.base import BaseEstimator, TransformerMixin

def extraiPronome(nome):
    return nome.split(',')[1].split('.')[0].strip()

# Remome colunas indesejadas no final do tratamento dos dados
class AtributosDesejados(BaseEstimator, TransformerMixin):
    def __init__(self, excluirName=True):
        self.excluirName = excluirName
    def fit(self, X, y=None):
        self.colunasIndesejadas = ['PassengerId', 'Name', 'Ticket', 'Age', 'Fare']
        if self.excluirName:
            self.colunasIndesejadas.append('Name')
        return self
    def transform(self, X, y=None):
        if 'Name' not in self.colunasIndesejadas:
            X['Name'] = X['Name'].apply(extraiPronome)
            
            # Trata pronome de tratamento
            X['Name'] = X['Name'].replace('Mlle', 'Miss')
            X['Name'] = X['Name'].replace(['Ms','Mme', 'Lady', 'the Countess', 'Dona'] , 'Mrs')           
            X['Name'] = X['Name'].replace(['the Countess', 'Dona', 
                                           'Capt', 'Col','Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer'], 'Other')


        # Define Cabin como a primeira letra
        X['Cabin'] = X['Cabin'].str[:1]
        ## Preenche com Z os faltantes
        X['Cabin'].fillna('Z', inplace=True)
        
        # Classe para o Sexo para drop 'Sex'
        X['Sex_Class'] = 0
        X.loc[ X['Sex'] == 'female', 'Sex_Class'] = 0
        X.loc[ X['Sex'] == 'male', 'Sex_Class'] = 1
        X['Sex_Class'] = X['Sex_Class'].astype(int)


        # Classes para as idades para drop 'Age'
        ## Preenche com a mediana os faltantes
        X['Age_Class'] = ''
        X['Age'].fillna(X['Age'].median(), inplace=True)
        
        X.loc[ X['Age'] < 16, 'Age_Class'] = 'children'
        X.loc[(X['Age'] >= 16) & (X['Age'] < 35), 'Age_Class'] = 'young'
        X.loc[(X['Age'] > 35) & (X['Age'] < 65), 'Age_Class'] = 'adult'
        X.loc[ X['Age'] >= 65, 'Age_Class'] = 'old'


        # Classes para tarifa para drop 'Fare'
        X['Fare_Class'] = ''
        ## Preenche com a mediana os faltantes
        X['Fare'].fillna(X['Fare'].median(), inplace=True)

        X.loc[ X['Fare'] < 20, 'Fare_Class'] = 'low_fare'
        X.loc[(X['Fare'] >= 20) & (X['Fare'] < 60), 'Fare_Class'] = 'median_fare'
        X.loc[(X['Fare'] >= 60) & (X['Fare'] < 120), 'Fare_Class'] = 'median_high_fare'
        X.loc[ X['Fare'] >= 120, 'Fare_Class'] = 'high_fare'


        # Cria característica FamilySize para classificar os tamanhos da família. FamilySize é removido ao final.
        X['FamilySize'] = 0
        X['FamilySize'] = X['SibSp'] + X['Parch'] + 1
        X['FamilySize'] = X['FamilySize'].astype(int)

        # X['Single'] = X['FamilySize'].map(lambda x: 1 if x==1 else 0)
        # X['Small_family'] = X['FamilySize'].map(lambda x: 1 if 2 <= x <=4 else 0)
        # X['Medium_family'] = X['FamilySize'].map(lambda x: 1 if 5 <= x <=7 else 0)
        # X['Large_family'] = X['FamilySize'].map(lambda x: 1 if 7 < x else 0)        
   

        # # Característica se é casada
        # X['Is_Married'] = 0
        # X['Is_Married'] = X['Name'].map(lambda x: 1 if x=='Mrs' else 0)   
   
        # Remove colunas indesejadas
        X = X.drop(self.colunasIndesejadas,axis=1)
        
        return X


In [176]:
from sklearn.base import BaseEstimator, TransformerMixin

class AtributosNumericos(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.colunasNumericas = X.select_dtypes(include='number').columns
        return self
    def transform(self, X, y=None):
        return X[self.colunasNumericas].to_numpy()


In [177]:
from sklearn.base import BaseEstimator, TransformerMixin

class AtributosCategoricos(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        self.colunasCategoricas = X.select_dtypes(include='object').columns
        return self
    def transform(self, X, y=None):
        return X[self.colunasCategoricas].to_numpy()


In [178]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline, FeatureUnion

trataAtributos = Pipeline([
    ('unecaracteristicas', FeatureUnion([
        ('pipenum', Pipeline([
            ('atributos_numericos', AtributosNumericos()),
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler())
        ])),
        ('pipecat', Pipeline([
            ('atributos_categoricos', AtributosCategoricos()),
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('encoder', OneHotEncoder(handle_unknown='ignore'))
        ]))
    ])),
])


In [179]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, cross_validate, RepeatedKFold
import numpy as np

pipetotal = Pipeline([
    ('atributosDesejados', AtributosDesejados()),
    ('trataAtributos', trataAtributos),
    ('classificador', RandomForestClassifier())
])

parametros = {
    'atributosDesejados__excluirName': [False],
    'classificador__max_depth': [4],
    #'classificador__max_features': [3, 4,'sqrt'],
    #'classificador__min_samples_split': [4],
    #'classificador__min_samples_leaf': [2],
    'classificador__n_estimators' : [50],
    'classificador__criterion': ['gini', 'entropy'],
    #'classificador__random_state': [0]
}

modelo = GridSearchCV(pipetotal, param_grid=parametros, n_jobs=-1)

scores = cross_validate(modelo, X, y, cv=RepeatedKFold())
scores['test_score'], np.mean(scores['test_score']), np.std(scores['test_score'])

(array([0.78212291, 0.82022472, 0.84269663, 0.80898876, 0.80337079,
        0.79329609, 0.83707865, 0.83146067, 0.79775281, 0.79775281,
        0.75977654, 0.8258427 , 0.84269663, 0.78089888, 0.86516854,
        0.75977654, 0.79775281, 0.80898876, 0.85393258, 0.85393258,
        0.82122905, 0.8258427 , 0.83146067, 0.75842697, 0.8258427 ,
        0.82681564, 0.7247191 , 0.88202247, 0.83146067, 0.78651685,
        0.77653631, 0.81460674, 0.82022472, 0.84269663, 0.80898876,
        0.82681564, 0.82022472, 0.78651685, 0.8258427 , 0.82022472,
        0.80446927, 0.81460674, 0.84269663, 0.84269663, 0.76966292,
        0.84916201, 0.83146067, 0.80898876, 0.74157303, 0.8258427 ]),
 0.8130337078651685,
 0.031443151618740436)

In [180]:
modelo.fit(X,y)
y_pred = modelo.predict(test)
result = test[['PassengerId']]
result['Survived'] = y_pred
result.to_csv('submission.csv',index=False)